In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
import pickle
from lightgbm.sklearn import LGBMClassifier

/usr/local/lib/python2.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [4]:
data_train = pd.read_csv("cleaned_2013_14", low_memory = False);
data_test = pd.read_csv("cleaned_2015", low_memory = False);

In [5]:
data_test = data_test.dropna()
data_test.columns == data_train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [6]:
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term_adj', 'zip_code']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)
x_test, y_test = split_data(data_test, cols_to_drop_training)

In [7]:
sm = SMOTE(random_state=1, ratio = 1.0)
x_train, y_train = sm.fit_sample(x_train_initial, y_train_initial)

In [8]:
x_train.shape, x_test.shape

((578050, 44), (346843, 44))

In [26]:
params = {}

params['LGBM__n_jobs'] = [5]
params['LGBM__learning_rate'] = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
params['LGBM__boosting_type'] = ['gbdt', 'dart']
params['LGBM__max_leaves'] = [2500, 5000, 7500, 10000, 12500, 14000, 16300]

In [27]:
# Use Pipeline instead of make_pipeline
pipe = Pipeline([('POLY', PolynomialFeatures(degree=2, include_bias=False)),
            ('LGBM',LGBMClassifier(objective = 'binary', metric = 'binary_logloss',
                                   n_jobs = 5, bagging_fraction = 0.7))])


# lgbm = pipe.fit(x_train, y_train)

# Instantiate GS
gs = GridSearchCV(pipe, params, verbose=10, cv=4)

In [28]:
# pipe.get_params().keys()

In [ ]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 84 candidates, totalling 336 fits
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=2500 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=2500, score=0.567896535976, total=  52.8s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=2500 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s


[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=2500, score=0.71423134411, total=  49.8s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=2500 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  9.4min remaining:    0.0s


[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500, score=0.567896535976, total=  50.1s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 10.6min remaining:    0.0s


[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500, score=0.71423134411, total=  49.0s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500 
[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500, score=0.679348427812, total=  49.1s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500 
[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=7500, score=0.539768323738, total=  48.7s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=10000 
[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=10000, score=0.567896535976, total=  50.1s
[CV] LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=10000 
[CV]  LGBM__learning_rate=0.001, LGBM__boosting_type=gbdt, LGBM__n_jobs=5,

[CV]  LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=12500, score=0.741474756422, total= 2.1min
[CV] LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=12500 
[CV]  LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=12500, score=0.698751660762, total=  50.8s
[CV] LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=12500 
[CV]  LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=12500, score=0.534599202834, total=  48.5s
[CV] LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=14000 
[CV]  LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=14000, score=0.612245180398, total=  49.3s
[CV] LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_jobs=5, LGBM__max_leaves=14000 
[CV]  LGBM__learning_rate=0.005, LGBM__boosting_type=gbdt, LGBM__n_j

In [11]:
print("Best parameters set found on training set:")
print()
print(gs.best_params_)
print()
print("Grid scores on training set:")
print()
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

Best parameters set found on training set:
()


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
train_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_train)[:, 1], 'true_val' : y_train}
test_preds_true = {'proba' : gs.best_estimator_.predict_proba(x_test)[:, 1], 'true_val' : y_test}

In [ ]:
AUC_train = roc_auc_score(train_preds_true['true_val'], train_preds_true['proba'])
AUC_test = roc_auc_score(test_preds_true['true_val'], test_preds_true['proba'])

In [ ]:
print('AUC Train: ', AUC_train);
print('AUC Test: ', AUC_test);

In [ ]:
# ## Predict
# train_predictions = lgbm.predict(x_train)
# test_predictions = lgbm.predict(x_test)

# ## Accuracy Score
# train_score = lgbm.score(x_train, y_train)
# test_score = lgbm.score(x_test, y_test)

# print("train accuracy: ", train_score)
# print("test accuracy: ", test_score)

# ## AUC
# train_predictions_class = np.where(train_predictions > 0.5, 1, 0)
# test_predictions_class = np.where(test_predictions > 0.5, 1, 0)

# print('Train ROC AUC: ', roc_auc_score(train_predictions_class, y_train))
# print('Test  ROC AUC: ', roc_auc_score(test_predictions_class, y_test))

In [ ]:
# from sklearn.metrics import roc_curve
# y_pred = lgbm.predict(x_test).ravel()
# fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# from sklearn.metrics import auc
# auc_lgbm = auc(fpr, tpr)

# plt.figure(1)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.plot(fpr, tpr, label='LGBM (area = {:.3f})'.format(auc_lgbm))
# plt.xlabel('False positive rate')
# plt.ylabel('True positive rate')
# plt.title('ROC curve')
# plt.legend(loc='best')
# plt.show()

In [ ]:
# Dump the tuned classifier with Pickle
pkl_filename = 'Tuned_LGBM.pkl'

# Open the file to save as pkl file
model_pkl = open(pkl_filename, 'wb')
pickle.dump(gs.best_estimator_, model_pkl)

# Close the pickle instances
model_pkl.close()